# Processing

- Importation 

In [1]:
import time
from collections import namedtuple
import numpy as np
import pandas as pd
from bert_serving.client import BertClient
from bert_serving.server import BertServer
from bert_serving.server.graph import PoolingStrategy
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA


- Reading data as a dataframe

In [2]:

data=pd.read_csv("path",encoding='utf-8',error_bad_lines=False, sep=';', header=None)

In [4]:
data.columns =['phrase','sujet']
data.head(10)

,phrase,sujet
0,Le sport est dépassement de soi. Le sport est ...,sports
1,Le football est le reflet de notre société. Re...,sports
2,Le sport est une évasion complète de la vie.,sports
3,"Le karaté c'est comme la vie, un art des renco...",sports
4,Je voudrais évoquer quelques points spécifiques,sports
5,"Le sport va chercher la peur pour la dominer, ...",sports
6,Le sport est dépassement de soi. Le sport est ...,sports
7,Pour important que soit le principe de concur...,sports
8,"Pratiqué avec sérieux, le sport n'a rien à voi...",sports
9,Quelles garanties avons-nous qu ils évoluero...,sports


In [5]:
shuffled = data.reindex(np.random.permutation(data.index))
shuffled.head(10)

,phrase,sujet
790,Le fait qu il ne définissait pas une véritabl...,"law, govt and politics"
679,"En insistant ainsi, en premier lieu sur la qu...","law, govt and politics"
1456,"On est si petit, le monde est si grand.\nQue s...",life
4631,L'ignorance est la science des dieux qui tuent.,science
4626,La seule vraie science est la connaissance des...,science
163,"""Faites en sorte que les vaincus puissent se f...",sports
1129,"Toutefois, la définition de la notion d inves...",finance
2915,J'ai écrit un roman qui tient en une seule phr...,religion
2584,"Si les gens connaissaient la mythologie, ils r...",religion
3900,Alterner systématiquement boisson non-alcooli...,health and fitness


- Class to number 

In [7]:
shuffled = shuffled.reindex(np.random.permutation(shuffled.index))
shuffled['LABEL'] = 0
shuffled.head(10)

,phrase,sujet,LABEL
2542,Une société d'athées inventerait aussitôt une ...,religion,0
1784,"Sur le Chemin, comme dans la vie, la sagesse n...",life,0
3202,Je m'appelle Richard. Qu'est ce que vous voule...,family and parenting,0
4472,"La science présente beaucoup de danger, mais i...",science,0
3110,Quand est-ce que j'aurais droit a un Noël norm...,family and parenting,0
1945,L'art de la photographie est d'immortaliser de...,art and entertainment,0
1049,"Ce systéme d équité, nous pouvons l envisager...",finance,0
3674,"Au niveau santé, je n'ai toujours pas eu d'enn...",health and fitness,0
662,Nous débattons d un projet de modernisation t...,"law, govt and politics",0
3702,Le respect avant tout et la santé pareille.,health and fitness,0


In [8]:
shuffled.loc[shuffled['sujet']== 'law, govt and politics', 'LABEL']=0
shuffled.loc[shuffled['sujet'] == 'society', 'LABEL']=1
shuffled.loc[shuffled['sujet'] == 'family and parenting','LABEL']=2
shuffled.loc[shuffled['sujet'] == 'religion','LABEL']=3
shuffled.loc[shuffled['sujet'] == 'life', 'LABEL']=4
shuffled.loc[shuffled['sujet'] == 'science', 'LABEL']=5
shuffled.loc[shuffled['sujet'] == 'finance', 'LABEL']=6
shuffled.loc[shuffled['sujet'] == 'health and fitnesss', 'LABEL']=7
shuffled.loc[shuffled['sujet']== 'sports', 'LABEL']=8
shuffled.loc[shuffled['sujet']== 'art and entertainment', 'LABEL']=9
shuffled.head(10)

,phrase,sujet,LABEL
2542,Une société d'athées inventerait aussitôt une ...,religion,3
1784,"Sur le Chemin, comme dans la vie, la sagesse n...",life,4
3202,Je m'appelle Richard. Qu'est ce que vous voule...,family and parenting,2
4472,"La science présente beaucoup de danger, mais i...",science,5
3110,Quand est-ce que j'aurais droit a un Noël norm...,family and parenting,2
1945,L'art de la photographie est d'immortaliser de...,art and entertainment,9
1049,"Ce systéme d équité, nous pouvons l envisager...",finance,6
3674,"Au niveau santé, je n'ai toujours pas eu d'enn...",health and fitness,0
662,Nous débattons d un projet de modernisation t...,"law, govt and politics",0
3702,Le respect avant tout et la santé pareille.,health and fitness,0


- lists of our label and text

In [9]:
subset_text = list(shuffled['phrase'].values)
subset_label = list(shuffled['LABEL'].values)
num_label = len(set(subset_label))

[3, 4, 2, 5, 2, 9, 6, 0, 0, 0, 3, 6, 2, 1, 4, 3, 4, 1, 0, 3, 8, 3, 0, 1, 8, 6, 2, 5, 0, 0, 8, 1, 4, 3, 5, 8, 6, 1, 2, 9, 9, 3, 5, 3, 6, 0, 5, 3, 6, 2, 8, 6, 0, 5, 4, 6, 8, 0, 1, 9, 0, 9, 4, 3, 3, 9, 6, 2, 8, 8, 1, 6, 0, 3, 0, 0, 5, 0, 0, 6, 3, 6, 8, 2, 4, 9, 2, 1, 2, 9, 8, 3, 9, 2, 8, 0, 2, 0, 0, 0, 0, 9, 4, 4, 6, 6, 2, 1, 0, 4, 4, 5, 0, 5, 0, 1, 4, 0, 5, 9, 4, 0, 8, 2, 4, 1, 2, 3, 1, 0, 6, 0, 3, 1, 3, 4, 8, 5, 9, 4, 2, 8, 8, 0, 8, 0, 6, 5, 9, 2, 4, 8, 8, 2, 2, 8, 1, 8, 5, 9, 6, 5, 3, 0, 5, 0, 0, 3, 0, 2, 0, 0, 4, 0, 4, 2, 9, 9, 9, 1, 8, 8, 2, 0, 1, 5, 4, 2, 5, 3, 3, 6, 1, 8, 2, 9, 2, 9, 0, 3, 9, 1, 5, 3, 9, 6, 3, 6, 3, 0, 0, 2, 5, 1, 1, 3, 4, 2, 5, 9, 9, 5, 2, 3, 0, 2, 1, 2, 5, 2, 9, 4, 1, 5, 6, 0, 5, 2, 1, 9, 1, 8, 1, 3, 6, 5, 0, 6, 1, 0, 9, 3, 0, 6, 0, 6, 2, 0, 0, 6, 5, 0, 9, 1, 6, 2, 0, 9, 2, 3, 3, 1, 8, 0, 6, 0, 4, 3, 3, 8, 3, 0, 9, 4, 0, 1, 1, 2, 3, 3, 2, 4, 0, 1, 4, 4, 2, 1, 8, 3, 5, 0, 5, 5, 1, 6, 5, 8, 8, 6, 4, 1, 2, 0, 3, 3, 6, 5, 9, 8, 5, 1, 6, 4, 3, 1, 5, 0, 0, 4, 0, 4, 0, 

In [11]:
print('min_seq_len: %d' % min(len(v.split()) for v in subset_text))
print('max_seq_len: %d' % max(len(v.split()) for v in subset_text))
print('unique label: %d' % num_label)

min_seq_len: 3
max_seq_len: 785
unique label: 9


# Using bert server

- We should lunch the server first using the folowing command :
         > bert_serving_start -pooling_layer -10 -model_dir="path"

In [12]:
# array to save our data 
B=[]

In [13]:
from bert_serving.client import BertClient
bc = BertClient(check_length=False)
B=bc.encode(subset_text)

In [17]:
B

array([[-0.4784688 ,  0.59450924, -0.30014026, ..., -0.46640718,
         0.12308042,  0.64219666],
       [-0.7514608 ,  0.62921876,  0.15271863, ..., -0.8905141 ,
         0.10648459,  0.43560937],
       [-0.713017  ,  0.6686921 ,  0.26946196, ..., -0.5205836 ,
         0.01374757,  0.2903203 ],
       ...,
       [-0.92430806,  0.536207  ,  0.17437717, ..., -0.42711276,
         0.06707807,  0.32152948],
       [-0.65371495,  0.68190503,  0.15354809, ..., -0.43683535,
         0.00304869,  0.10963322],
       [-0.3907939 ,  0.23548323, -0.24272089, ..., -0.5956548 ,
         0.15462498,  0.3665693 ]], dtype=float32)

# Saving results

Save post (text) results in a csv file

In [1]:
import csv 
import codecs

def writterCsv(B,pathoutput):
    with codecs.open(str(pathoutput),"w+", encoding="utf8") as f:
         for liste in B:
            for i in liste:
                f.write('{0};'.format(i))
            f.write('\n')

In [19]:
path="C:/User/Desktop/bert-master/bert/output_bert_layer10.csv"
writterCsv(B,path)

Save label in a csv file

In [20]:
import csv 
import codecs

def writterCsv(B,pathoutput):
    with codecs.open(str(pathoutput),"w+", encoding="utf8") as f:
        for liste in B:
            f.write('{0};'.format(liste))
            f.write('\n')
path="C:/Users/Desktop/bert-master/bert/layer10.csv"
writterCsv(subset_label,path)

Note : We should link manually between our posts and our labels in order to avoid any problem
    - We just need to copy our labels (with the same order) and past them in the 769 column of post's file 
    - Then we replace the fiste line (in post's file ) by p1 to p768 and for 789 column we put it by LABEL 